In [180]:
import pandas as pd
import numpy as np

In [181]:
df = pd.read_csv("./UH_2023/UH_2023_TRAIN.txt", sep = "|")
df = df.astype({"CAMPAÑA": str, "ID_FINCA": str, "ID_ZONA": str, "ID_ESTACION": str, "VARIEDAD": str, "MODO": str, "TIPO": str, "COLOR": str, "SUPERFICIE": str})

In [182]:
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
encoder = ce.OrdinalEncoder(cols=["ID_FINCA", "ID_ZONA", "ID_ESTACION", "VARIEDAD", "COLOR", "TIPO", "MODO"])

In [184]:
df_temporal = df.drop(axis = 1, columns = ["CAMPAÑA","PRODUCCION", "ALTITUD"])

df_encoded = encoder.fit_transform(df_temporal)

train = df_encoded[(df["PRODUCCION"] != 0) & (df_encoded["SUPERFICIE"] != "0.0")]

to_predict = df_encoded[~((df["PRODUCCION"] != 0) & (df_encoded["SUPERFICIE"] != 0))]
l_encoder = LabelEncoder()
train["SUPERFICIE"] = l_encoder.fit_transform(train["SUPERFICIE"])

to_predict = to_predict.drop(axis = 1, columns = ["SUPERFICIE"])

C:\Users\xavid\AppData\Local\Temp\ipykernel_9248\3463993660.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["SUPERFICIE"] = l_encoder.fit_transform(train["SUPERFICIE"])


In [185]:
df_supcat = train.groupby(by='SUPERFICIE').count().reset_index()
segur_train = df_supcat[df_supcat["TIPO"] == 1]["SUPERFICIE"]
train_nou = train[train['SUPERFICIE'].isin(list(segur_train))]

In [186]:
train_a_splitejar = train.drop(index=train_nou.index)

In [187]:
X_train_a_splitejar = train_a_splitejar.drop(axis = 1, columns = ["SUPERFICIE"])
y_train_a_splitejar = train_a_splitejar["SUPERFICIE"]

In [188]:
X_train_nou = train_nou.drop(axis = 1, columns = ["SUPERFICIE"])
y_train_nou = train_nou["SUPERFICIE"]

In [189]:
X_train_splitted, X_test, y_train_splitted, y_test = train_test_split(X_train_a_splitejar, y_train_a_splitejar, test_size=0.35, random_state=3, stratify = train_a_splitejar["SUPERFICIE"])

In [190]:
X_train = pd.concat([X_train_splitted, X_train_nou])
y_train = pd.concat([y_train_splitted, y_train_nou])

In [191]:
len(X_test) + len(X_train) == len(train)

True

In [194]:
import xgboost as xgb
params_model = {'colsample_bytree': 0.4, 'gamma': 0.0, 'learning_rate': 0.15, 'max_depth': 100, 'min_child_weight': 1, 'n_estimators': 2000}
model = xgb.XGBClassifier(**params_model)
model.fit(X_train,y_train)

0.7258566978193146

In [205]:
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.7334905660377359

In [195]:
y_test_dec_gt = l_encoder.inverse_transform(y_test)
y_predicted = l_encoder.inverse_transform(model.predict(X_test))

In [196]:
y_test_dec_gt = y_test_dec_gt.astype(np.float)
y_predicted = y_predicted.astype(np.float)

In [197]:
from sklearn.metrics import r2_score
r2_score(y_test_dec_gt, y_predicted)

0.559937852007629

In [202]:
len(y_train)

2120

In [204]:
model.predict_proba(X_test)

array([[6.8190414e-04, 3.8513844e-04, 1.7557872e-04, ..., 5.1132516e-05,
        1.1260376e-04, 5.8873522e-04],
       [1.4687682e-04, 4.1054318e-05, 9.5589610e-05, ..., 1.7975106e-04,
        3.6124216e-05, 8.1145583e-05],
       [3.8297463e-04, 1.6848360e-04, 1.4259036e-04, ..., 2.7710092e-04,
        1.7916671e-03, 4.3455581e-04],
       ...,
       [1.2515013e-03, 3.2755014e-04, 1.6364275e-04, ..., 1.5561598e-04,
        5.9414026e-04, 2.9936555e-04],
       [2.1049769e-04, 5.8837293e-05, 2.6087352e-04, ..., 2.5761157e-04,
        5.1771684e-05, 1.1629438e-04],
       [1.4517110e-04, 1.2535493e-04, 3.9822314e-04, ..., 2.7865608e-04,
        1.0341343e-04, 4.9287215e-04]], dtype=float32)